# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

In [3]:
len(CORD19_CSV['doi'])

77448

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

In [5]:
#delete Nans
doi_counted = doi.value_counts()
doi_counted

10.1016/j.dsx.2020.04.012        2
10.31729/jnma.5498               2
10.1016/j.addr.2009.07.001       1
10.1053/j.gastro.2020.08.039     1
10.1101/2020.07.22.216275        1
                                ..
10.7759/cureus.12177             1
10.1186/s13063-020-04912-z       1
10.1101/2021.01.28.21250129      1
10.1016/s0140-6736(03)13582-9    1
10.1371/journal.pmed.1001444     1
Name: doi, Length: 74302, dtype: int64

In [6]:
def AffiliationsFromScopusByDOI(client, doi):
    """obtain addiotional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

In [7]:
## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()
## Initialize client
client = ElsClient(config['apikey'])
# https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4

In [8]:
df_current_extra_info = pd.read_pickle('extra_info_CS5099.pkl')
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
58215,NaN,"{'srctype': 'j', 'eid': '2-s2.0-85085192672', ..."
58216,"{'affiliation-city': 'Sao Paulo', 'affilname':...","{'srctype': 'j', 'eid': '2-s2.0-79952191861', ..."
58217,None,None
58218,"[{'affiliation-city': 'Silchar', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85098660877', ..."


In [9]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

58220

In [ ]:
%%time
i = len_df_current_extra_info
dict_new_extra_info = dict()
len_dois = len(doi_counted)
while i < len_dois:
    print("Position: " + str(i) + " -> " +  doi_counted.index[i])
    dict_new_extra_info[i] = AffiliationsFromScopusByDOI(client, doi_counted.index[i])
    i = i + 1 

Position: 58220 -> 10.1002/jmv.26227
Position: 58221 -> 10.3389/fbioe.2019.00106
Position: 58222 -> 10.1186/1471-2105-11-210
Position: 58223 -> 10.1016/bs.mie.2020.12.020
Position: 58224 -> 10.1016/j.dsx.2020.05.050
Position: 58225 -> 10.1155/2020/5098598
Position: 58226 -> 10.3390/ma13081997
Position: 58227 -> 10.7150/thno.20151
Position: 58228 -> 10.1016/j.chaos.2020.110023
Position: 58229 -> 10.1016/j.radi.2020.07.015
Position: 58230 -> 10.1016/j.emcchi.2004.05.002
Position: 58231 -> 10.3390/genes12010030
Position: 58232 -> 10.1038/s41598-020-72611-5
Position: 58233 -> 10.3390/md19010028
Position: 58234 -> 10.3201/eid2313.170348
Position: 58235 -> 10.14309/ajg.0000000000000788
Position: 58236 -> 10.3332/ecancer.2020.ed102
Position: 58237 -> 10.1016/j.meegid.2020.104212
Position: 58238 -> 10.1016/j.pt.2015.01.002
Position: 58239 -> 10.1371/journal.pcbi.1005697
Position: 58240 -> 10.3382/ps/pex093
Position: 58241 -> 10.1016/j.jinf.2020.03.019
Position: 58242 -> 10.1016/j.virusres.2015

Position: 58404 -> 10.3390/v11110993
Position: 58405 -> 10.1080/22221751.2020.1738279
Position: 58406 -> 10.1016/j.resuscitation.2020.09.009
Position: 58407 -> 10.1007/s40124-020-00216-7
Position: 58408 -> 10.1007/s10753-020-01239-4
Position: 58409 -> 10.1016/j.celrep.2015.09.048
Position: 58410 -> 10.3389/fimmu.2020.575504
Position: 58411 -> 10.3390/s21020660
Position: 58412 -> 10.1016/j.jinf.2020.05.017
Position: 58413 -> 10.1213/xaa.0000000000001253
Position: 58414 -> 10.1371/journal.pone.0148986
Position: 58415 -> 10.1111/ans.16069
Position: 58416 -> 10.1097/jom.0000000000002076
Position: 58417 -> 10.1016/j.vetmic.2020.108933
Position: 58418 -> 10.1186/s13063-020-04453-5
Position: 58419 -> 10.1007/978-3-030-51372-6_16
Position: 58420 -> 10.1371/journal.pone.0178241
Position: 58421 -> 10.1016/j.bspc.2020.102365
Position: 58422 -> 10.1373/clinchem.2011.161968
Position: 58423 -> 10.1292/jvms.18-0249
Position: 58424 -> 10.1016/j.ypmed.2020.106186
Position: 58425 -> 10.1101/2020.11.25.2

Position: 58586 -> 10.1371/journal.pone.0081090
Position: 58587 -> 10.1016/s2213-2600(20)30459-8
Position: 58588 -> 10.1038/s41467-020-19761-2
Position: 58589 -> 10.1136/bmjopen-2020-038827
Position: 58590 -> 10.1006/jmbi.1999.2860
Position: 58591 -> 10.1007/s10900-020-00870-4
Position: 58592 -> 10.12659/aot.926992
Position: 58593 -> 10.1128/mbio.00056-13
Position: 58594 -> 10.1101/2020.06.29.20142349
Position: 58595 -> 10.1534/genetics.120.303596
Position: 58596 -> 10.3390/biom10091223
Position: 58597 -> 10.1038/srep44190
Position: 58598 -> 10.1055/a-1181-8391
Position: 58599 -> 10.3390/e22101139
Position: 58600 -> 10.1007/s11883-020-00895-z
Position: 58601 -> 10.1016/j.vaccine.2018.10.005
Position: 58602 -> 10.1186/s40900-020-00248-9
Position: 58603 -> 10.1111/hae.14160
Position: 58604 -> 10.7759/cureus.8622
Position: 58605 -> 10.1016/j.gastrohep.2020.05.003
Position: 58606 -> 10.1111/1759-7714.13682
Position: 58607 -> 10.1016/j.bbapap.2007.09.013
Position: 58608 -> 10.1007/s11064-01

Position: 58769 -> 10.3390/microorganisms8081250
Position: 58770 -> 10.1101/2020.05.17.20105023
Position: 58771 -> 10.1007/s10654-016-0183-1
Position: 58772 -> 10.1016/j.hrmr.2020.100768
Position: 58773 -> 10.3390/molecules25245934
Position: 58774 -> 10.5853/jos.2020.01550
Position: 58775 -> 10.1007/s11612-020-00540-y
Position: 58776 -> 10.1007/978-1-4614-5401-4_17
Position: 58777 -> 10.1016/j.jcv.2020.104579
Position: 58778 -> 10.3390/ijms21093179
Position: 58779 -> 10.1007/978-3-030-45439-5_48
Position: 58780 -> 10.2144/fsoa-2020-0157
Position: 58781 -> 10.1016/j.sleep.2020.12.041
Position: 58782 -> 10.1093/jssam/smx032
Position: 58783 -> 10.1007/s10993-020-09571-z
Position: 58784 -> 10.1007/s11908-011-0223-7
Position: 58785 -> 10.1128/mbio.01430-14
Position: 58786 -> 10.3168/jds.2011-5085
Position: 58787 -> 10.1186/1297-9716-45-20
Position: 58788 -> 10.1101/054072
Position: 58789 -> 10.1002/ehf2.12937
Position: 58790 -> 10.1007/s00267-020-01293-6
Position: 58791 -> 10.3390/cancers12

Position: 58953 -> 10.1186/s12864-020-06829-y
Position: 58954 -> 10.1038/mtm.2014.5
Position: 58955 -> 10.1007/s10620-021-06842-5
Position: 58956 -> 10.1186/1471-2172-6-2
Position: 58957 -> 10.3390/ijerph17155603
Position: 58958 -> 10.1371/journal.pone.0183544
Position: 58959 -> 10.1186/s13054-020-03183-z
Position: 58960 -> 10.1007/s11033-020-05958-7
Position: 58961 -> 10.3389/fpsyt.2020.605140
Position: 58962 -> 10.1093/jpepsy/jsaa094
Position: 58963 -> 10.1002/lary.24348
Position: 58964 -> 10.3389/fpsyg.2020.02104
Position: 58965 -> 10.1101/2021.01.16.21249939
Position: 58966 -> 10.1093/jalm/jfaa195
Position: 58967 -> 10.1007/s00103-020-03249-x
Position: 58968 -> 10.1007/s11192-009-0051-9
Position: 58969 -> 10.1007/s38311-020-0308-8
Position: 58970 -> 10.1016/s1674-8301(11)60013-4
Position: 58971 -> 10.1101/2020.07.23.218529
Position: 58972 -> 10.1046/j.1525-1497.2001.0160s1023.x
Position: 58973 -> 10.3390/ijms21207475
Position: 58974 -> 10.1016/j.frl.2020.101703
Position: 58975 -> 1

...

Position: 64050 -> 10.1101/2020.08.25.265561
Position: 64051 -> 10.1007/978-3-030-52200-1_25
Position: 64052 -> 10.3390/polym12102341
Position: 64053 -> 10.1101/2020.09.26.20202051
Position: 64054 -> 10.1016/j.ajem.2020.10.036
Position: 64055 -> 10.1016/j.intimp.2020.107239
Position: 64056 -> 10.1097/01.mlg.0000249922.37381.1e
Position: 64057 -> 10.1007/s10393-018-1338-1
Position: 64058 -> 10.1016/j.vetmic.2003.07.010
Position: 64059 -> 10.2147/jhl.s266491
Position: 64060 -> 10.1186/s13317-020-00141-1
Position: 64061 -> 10.1371/journal.pone.0244373
Position: 64062 -> 10.1007/978-3-030-44914-8_25
Position: 64063 -> 10.1016/j.socscimed.2020.113449
Position: 64064 -> 10.1016/j.bsheal.2019.02.001
Position: 64065 -> 10.15252/embr.201540509
Position: 64066 -> 10.1101/2020.07.03.20146159
Position: 64067 -> 10.1080/07391102.2020.1777901
Position: 64068 -> 10.1101/2020.04.09.20056325
Position: 64069 -> 10.3390/ijms21176258
Position: 64070 -> 10.3390/molecules25225409
Position: 64071 -> 10.1148/

Position: 64231 -> 10.1007/978-3-030-52237-7_36
Position: 64232 -> 10.1101/2021.01.29.21250655
Position: 64233 -> 10.1016/j.arbres.2020.07.005
Position: 64234 -> 10.1016/j.ajic.2011.08.020
Position: 64235 -> 10.1007/s13238-010-0101-3
Position: 64236 -> 10.1016/j.vaccine.2016.10.090
Position: 64237 -> 10.1016/j.immuni.2020.07.019
Position: 64238 -> 10.1016/j.jcv.2020.104581
Position: 64239 -> 10.1016/j.red.2020.08.004
Position: 64240 -> 10.1136/bmjopen-2020-043577
Position: 64241 -> 10.1371/journal.pone.0166260
Position: 64242 -> 10.1007/s12144-020-01141-0
Position: 64243 -> 10.1088/0957-0233/19/6/065204
Position: 64244 -> 10.1101/2020.12.21.20248605
Position: 64245 -> 10.7150/jca.24744
Position: 64246 -> 10.1016/0167-5877(95)01000-9
Position: 64247 -> 10.1016/j.jcpa.2010.04.007
Position: 64248 -> 10.1111/joim.13160
Position: 64249 -> 10.1007/s11239-020-02270-w
Position: 64250 -> 10.1093/qjmed/hcaa206
Position: 64251 -> 10.1093/labmed/lmaa067
Position: 64252 -> 10.3390/healthcare8040578

Position: 64412 -> 10.1007/s007050050680
Position: 64413 -> 10.1109/tbme.2019.2910551
Position: 64414 -> 10.1186/s13071-020-04465-9
Position: 64415 -> 10.1186/s12911-020-01336-2
Position: 64416 -> 10.1016/j.edumed.2020.11.010
Position: 64417 -> 10.1016/j.diagmicrobio.2018.11.014
Position: 64418 -> 10.1080/07391102.2020.1817786
Position: 64419 -> 10.1111/j.1472-8206.2010.00844.x
Position: 64420 -> 10.1101/2020.08.24.264895
Position: 64421 -> 10.1007/s42438-020-00168-7
Position: 64422 -> 10.1007/978-3-030-49418-6_11
Position: 64423 -> 10.1186/s10020-020-00224-9
Position: 64424 -> 10.7759/cureus.8632
Position: 64425 -> 10.1016/j.ecolind.2020.106705
Position: 64426 -> 10.1128/jcm.01243-20
Position: 64427 -> 10.1016/j.indcrop.2020.112503
Position: 64428 -> 10.1186/s40545-020-00254-y
Position: 64429 -> 10.1016/j.psychres.2020.113692
Position: 64430 -> 10.1186/s13054-020-2823-9
Position: 64431 -> 10.1016/j.envres.2021.110777
Position: 64432 -> 10.1007/978-3-030-47426-3_62
Position: 64433 -> 1

Position: 64594 -> 10.1016/j.clinmicnews.2010.03.003
Position: 64595 -> 10.1016/j.eng.2020.05.016
Position: 64596 -> 10.1007/s00259-020-04968-8
Position: 64597 -> 10.3390/jcm9123935
Position: 64598 -> 10.1007/s13187-020-01870-7
Position: 64599 -> 10.1136/bmjgh-2020-003276
Position: 64600 -> 10.1038/s41598-018-31607-y
Position: 64601 -> 10.1136/bmjoq-2020-001014
Position: 64602 -> 10.1016/j.sjbs.2020.11.065
Position: 64603 -> 10.1128/jcm.00512-20
Position: 64604 -> 10.1126/sciadv.aav4580
Position: 64605 -> 10.1007/s40199-020-00369-2
Position: 64606 -> 10.3390/ijerph17218272
Position: 64607 -> 10.1183/23120541.00718-2020
Position: 64608 -> 10.1007/s00540-019-02702-9
Position: 64609 -> 10.1038/s41431-020-0625-9
Position: 64610 -> 10.1016/j.bbadis.2010.07.016
Position: 64611 -> 10.1186/s40673-020-00118-w
Position: 64612 -> 10.1093/bioinformatics/bti404
Position: 64613 -> 10.1016/j.taap.2020.115182
Position: 64614 -> 10.1021/acs.jpcb.0c10637
Position: 64615 -> 10.1186/s40814-020-00714-x
Pos

Position: 64774 -> 10.3389/fmicb.2016.01886
Position: 64775 -> 10.1093/mnrasl/slaa144
Position: 64776 -> 10.1007/s00520-020-05965-1
Position: 64777 -> 10.3389/fmicb.2018.03174
Position: 64778 -> 10.1016/j.virol.2004.09.006
Position: 64779 -> 10.1101/2021.01.21.21250268
Position: 64780 -> 10.1016/j.lanwpc.2020.100056
Position: 64781 -> 10.1038/srep26748
Position: 64782 -> 10.1186/s12967-020-02427-4
Position: 64783 -> 10.1016/j.watres.2013.07.041
Position: 64784 -> 10.1111/j.1939-1676.2011.0726.x
Position: 64785 -> 10.1007/s00521-020-05451-z
Position: 64786 -> 10.1007/s40119-020-00191-6
Position: 64787 -> 10.1038/s41522-019-0095-4
Position: 64788 -> 10.3390/ijerph17124279
Position: 64789 -> 10.1101/2020.09.18.296657
Position: 64790 -> 10.3389/fpubh.2020.562024
Position: 64791 -> 10.1007/s12038-017-9676-7
Position: 64792 -> 10.1016/j.promfg.2020.05.114
Position: 64793 -> 10.1007/s00431-020-03747-9
Position: 64794 -> 10.1016/j.lfs.2020.118408
Position: 64795 -> 10.1029/2020gl090244
Positio

In [ ]:
# dict_new_extra_info_saver = dict()
# i = len_df_current_extra_info
# while i < 47683:
#     #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
#     dict_new_extra_info_saver[i] = dict_new_extra_info[i]
#     i = i + 1 

# dict_new_extra_info = dict_new_extra_info_saver

In [ ]:
dict_new_extra_info

In [ ]:
df_new_extra_info = pd.DataFrame(dict_new_extra_info)

In [ ]:
df_new_extra_info

In [ ]:
df_new_extra_transposed = df_new_extra_info.T
df_new_extra_transposed

In [ ]:
df_combined_extra_info = pd.concat([df_current_extra_info, df_new_extra_transposed],ignore_index=True)

In [ ]:
df_combined_extra_info.to_pickle('extra_info_CS5099.pkl')
df_combined_extra_info

In [ ]:
entry = AffiliationsFromScopusByDOI(client, '10.1086/605034')
entry

In [ ]:
print (json.dumps(entry, ensure_ascii=False, indent=4))

In [ ]:
df_test = pd.DataFrame.from_dict(entry['affiliation'], orient='columns')
df_test

In [ ]:
entry['coredata']

In [ ]:
## ScienceDirect (full-text) document example using DOI
doi_doc = FullDoc(doi = '10.1016/S1525-1578(10)60571-5')
if doi_doc.read(client):
    print ("doi_doc.title: ", doi_doc.title)
    doi_doc.write()   
else:
    print ("Read document failed.")
doi_doc